In [53]:
def readDataFromTxt(filePath):
    text = []
    fh = open(filePath,encoding= 'utf8').readlines()
    for line in fh:
        line.replace("\n", '')
        text.append(line)
    return text

In [54]:
def deleteAlphabet(word):
    newWord = ""
    index = np.random.randint(0,len(word))
    if index == 0:
        newWord = word[1:]
    elif index == len(word) -1 :
        newWord = word[:len(word)-1]
    else:
        newWord = word[:index] + word[index+1:]
    return newWord  

In [55]:
def changeOrder(word):
    index = np.random.randint(0,len(word))
    secIndex = np.random.randint(0,len(word))
    while (index == secIndex):
        secIndex = np.random.randint(0,len(word))
    strlist = list(word)

    first = ''
    sec = ''
    if strlist[index].isupper():
        sec = strlist[secIndex].upper()
    if strlist[secIndex].isupper():
        first = strlist[index].upper()
    else:
        first = strlist[index].lower()
    strlist[index],strlist[secIndex] = sec,first
    return "".join(strlist)

In [56]:
import string
import random
def insertAlphabet(word):
    index = np.random.randint(0,len(word))
    pre = word[:index]
    post = word[index:]
    randomAlphabet = random.choice(string.ascii_lowercase)
    if word[index].isupper():
        randomAlphabet = randomAlphabet.upper()
        word[index].lower()
    return (pre + randomAlphabet + post)

In [57]:
def substituteAlphabet(word):
    randomAlphabet = random.choice(string.ascii_lowercase)
    index = np.random.randint(0,len(word))
    pre = word[:index]
    post = word[index+1:]
    if word[index][0].isupper():
        randomAlphabet = randomAlphabet.upper()
    if index == 0:
        pre = ''
    elif index == len(word) -1 :
        post = ''
    return (pre + randomAlphabet + post)

In [58]:
from PyDictionary import PyDictionary
import re
import numpy as np
dictionary = PyDictionary()

def syncWordGeneration(word):
    generatedWord = word
    while(dictionary.meaning(generatedWord,True) is not None): 
        operationNumb = np.random.randint(1,5)
        if operationNumb == 1:
            generatedWord  = deleteAlphabet(word)
        elif operationNumb == 2:
            generatedWord = changeOrder(word)
        elif operationNumb == 3:
            generatedWord = insertAlphabet(word)
        else:
            generatedWord = substituteAlphabet(word)
        print("Generated Word:",generatedWord)
    return generatedWord

In [59]:
import numpy as np
import time
def syncDataListGeneration(dataArray):
    generatedDataList = list()
    for data in dataArray:
        dataList = re.findall("[a-zA-Z]*’?[a-zA-Z]+|[\W]",data)
        orgWords = list(filter(lambda x : x != " ",dataList))
        words = re.findall("[a-zA-Z]*’?[a-zA-Z]+",data)
        randoms = np.random.randint(0,len(words), int(len(words)/20) +1)
        if len(randoms) != 0:
            for index in randoms:
                while(len(words[index]) == 1):
                    index = np.random.randint(0,len(words))
                start = time.time()
                print("current word: ",words[index])
                generatedWord = syncWordGeneration(words[index])
                print("It takes : ", time.time() - start, " generated word: ", generatedWord)
                
                if words[index] not in orgWords:
                    print(orgWords)
                else :
                    orgIndex = orgWords.index(words[index])
                    orgWords[orgIndex] = generatedWord
        dataString = " ".join(orgWords)

        for i in re.findall('[\W]\s|\s[\W]',dataString):
            dataString = dataString.replace(i,i.strip())
        finalList = dataString.split()

        for id in range(1,int(len(finalList)/10)+1):
            finalList.insert(10*id,'\n')

        print(finalList)
        
        generatedDataList.append(" ".join(finalList))

    return generatedDataList

In [60]:
from PIL import Image, ImageFont

def addSaltAndPepperNoise(image, amount):
    output = np.copy(np.array(image))
    
    salt =  np.random.randint(0,255)
    pepper = np.random.randint(0,255)

    # add salt
    nb_salt = np.ceil(amount * output.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(nb_salt)) for i in output.shape]
    output[coords] = salt

    # add pepper
    nb_pepper = np.ceil(amount* output.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(nb_pepper)) for i in output.shape]
    output[coords] = pepper
   
    return Image.fromarray(output)

In [61]:
from PIL import ImageDraw,ImageFilter
from random import randint, seed

def addBackgroundNoise(image,draw):
    
    # Let's have repeatable, deterministic randomness
    seed(37)

    # Generate a basic random colour, random RGB values 10-245
    R, G, B = randint(10,245), randint(10,245), randint(10,245),

    # Draw a random number of circles between 40-120
    cmin = randint(50, 100)
    cmax = randint(90,150)
    for _ in range(cmin,cmax):
        # Choose RGB values for this circle, somewhat close (+/-10) to basic RGB
        r = R + randint(-10,100)
        g = G + randint(-10,100)
        b = B + randint(-10,100)
        diam = randint(5,11)
        x, y = randint(0,1800), randint(0,500)
        draw.ellipse([x,y,x+diam,y+diam], fill=(r,g,b))
    # Blur the background a bit
    res = image.filter(ImageFilter.BoxBlur(2))
    draw = ImageDraw.Draw(res)
    return res

In [62]:
from PIL import Image, ImageFont,ImageFilter

def addNoise(image,draw):
    img = image
    rand = random.randint(0,6)
    if rand == 0:
        amount = np.random.randint(0.4,2)
        img = addSaltAndPepperNoise(image,amount)
    elif rand ==1 :
        img = addBackgroundNoise(image,draw)
    elif rand == 2:
        img = image.filter(ImageFilter.GaussianBlur(1))
    elif rand == 3:
        img = image.filter(ImageFilter.EMBOSS)
    elif rand == 4:
        img = image.filter(ImageFilter.GaussianBlur(1))
        img = image.filter(ImageFilter.EMBOSS)  
    return img

In [63]:
from PIL import Image, ImageFont, ImageDraw

fontTypes = ['Calibri','ENGR','Arial','BRUSHSCI','Bahnschrift','Candara','LATINWD','CASTELAR','Corbel','Ebrima','BROADW','Gadugi','BAUHS93',
'Georgia','Verdana','ITCBLKAD','CENTURY','COLONNA','COOPBL','CURLZ___','FELIXTI','FTLTLT','FORTE','FREESCPT','Gabriola',
'GLECB','HARLOWSI','HARNGTON','impact','IMPRISHA','INFROMAN','Inkfree','javatext','JOKERMAN',
'ITCKRIST','LHANDW']

fontSizes = [28,18,28,38,30,28,16,22,31,31,26,28,27,31,26,32,32,35,28,32,24,32,34,50,32,38,33,29,33,31,33,32,30,30,28,24]

def saveImage(data,numb,path):
    rand = random.randint(0,len(fontTypes)-1)
    fontFamily = "C:\Windows\Fonts\/"+ fontTypes[rand] + ".ttf"
    font = ImageFont.truetype(fontFamily,fontSizes[rand])
    img = Image.new('RGB', (1000, 200), color = (221, 227, 218))
    draw = ImageDraw.Draw(img)
    
    draw.text((20,20), data, fill=(0,0,0), font = font, encoding= 'utf8')
    img = addNoise(img, draw)

    angle = random.randint(0,20)
    if rand%3 == 0:
        img = img.rotate(angle,expand = 1)
    name = "generatedData"+str(numb)+".png"
    img.save(path+name)

In [64]:
def writeDataToFile(file, data):
        file = open(file,"a")
        for instance in data:
            file.write(instance)
            file.write("\n")
        file.close()    

In [65]:
text = readDataFromTxt("D:\ASSOC\englishSpellingCorrection\Dataset\Script\sample1.txt")

imagePath = "D:\ASSOC\englishSpellingCorrection\Dataset\Generated_Images\\"
filePath = "D:\ASSOC\englishSpellingCorrection\Dataset\Label\\"
generatedData = syncDataListGeneration(text)
writeDataToFile(filePath + 'orginalData.txt',text)  
writeDataToFile(filePath + 'generatedData.txt',generatedData)
count = 1
for data in generatedData:
    saveImage(data,count,imagePath)
    count += 1

current word:  Hello
Generated Word: Hmello
It takes :  1.7398414611816406  generated word:  Hmello
['Hmello,', 'Hi,', 'Hao']
